In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso, LassoCV, RidgeCV, Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import numpy as np

### Carregar e visualizar os dados - Regressão

In [2]:
# Carregar os dados - Regressão
df = pd.read_csv('./dataset/dataset_colesterol.csv')

In [3]:
# Visualiar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [4]:
# Visualizar DataFrame
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23
5,6,B,Não,Baixo,68,66.8,170,185.31
6,7,A,Sim,Baixo,60,117.3,181,289.33
7,8,O,Sim,Moderado,35,86.9,174,216.48
8,9,O,Não,Baixo,62,81.3,166,235.30
9,10,B,Sim,Alto,44,32.7,165,97.79


In [5]:
# Ajustar DataFrame
df.drop('Id', axis=1, inplace=True)

In [6]:

# Aplicar OneHotEncoding nas variáveis categoricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [7]:
# DataFrame atualizado
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,0,0,1,0,0,1,0,1,0
1,68,105.0,184,236.98,1,0,0,0,1,0,0,0,1
2,25,64.8,180,161.79,0,0,0,1,1,0,1,0,0
3,43,120.2,167,336.24,1,0,0,0,1,0,1,0,0
4,79,88.5,175,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,68.1,166,206.81,0,1,0,0,0,1,0,0,1
996,51,47.7,170,128.03,0,0,0,1,1,0,1,0,0
997,39,85.5,176,211.14,0,1,0,0,1,0,0,1,0
998,61,91.2,161,284.53,0,1,0,0,0,1,0,1,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        1000 non-null   int64  
 1   Peso                         1000 non-null   float64
 2   Altura                       1000 non-null   int64  
 3   Colesterol                   1000 non-null   float64
 4   Grupo Sanguíneo_A            1000 non-null   uint8  
 5   Grupo Sanguíneo_AB           1000 non-null   uint8  
 6   Grupo Sanguíneo_B            1000 non-null   uint8  
 7   Grupo Sanguíneo_O            1000 non-null   uint8  
 8   Fumante_Não                  1000 non-null   uint8  
 9   Fumante_Sim                  1000 non-null   uint8  
 10  Nível de Atividade_Alto      1000 non-null   uint8  
 11  Nível de Atividade_Baixo     1000 non-null   uint8  
 12  Nível de Atividade_Moderado  1000 non-null   uint8  
dtypes: float64(2), int6

### Treinar Modelo de Regressão com Lasso (L1)

In [9]:
# Separar X e y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [10]:
# Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [11]:
# Treinar o Modelo de regressão linear múltipla com Lasso
# Quanto maior o alpha, maior a penalização e mais coeficientes tendem a ser reduzios a zero
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [12]:
# Mostrar importância de Features
def importancia_features(modelo):
    importance = np.abs(modelo.coef_)
    print('Importância das Features')
    for i, feature in enumerate(modelo.feature_names_in_):
        print(f'{feature}: {importance[i]}')

In [13]:
# Importância das Features - Lasso
importancia_features(model_lasso)

Importância das Features
Idade: 0.017349176463551177
Peso: 2.4911793204672508
Altura: 2.20863191913672
Grupo Sanguíneo_A: 0.05063677331183119
Grupo Sanguíneo_AB: 1.032901896958969
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.6422787796520806
Fumante_Sim: 0.0
Nível de Atividade_Alto: 1.485309859678915
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.17448169861080404


In [14]:
# Função para a avaliar performance
def performace_regressao(modelo, X_test, y_test):
    # Faz a predição com o modelo no conjunto de testes
    y_pred = modelo.predict(X_test)
    # Avaliar desempenho
    return root_mean_squared_error(y_test, y_pred)

In [15]:
# Performance Regressão com Lasso
performace_regressao(model_lasso, X_test, y_test)

8.96623471437868

### Treinar com LassoCV

In [19]:
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X, y)

,eps,0.001
,n_alphas,'deprecated'
,alphas,"[0.1, 0.5, ...]"
,fit_intercept,True
,precompute,'auto'
,max_iter,1000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [20]:
# Importância das Features - LassoCV
importancia_features(model_lasso_cv)

Importância das Features
Idade: 0.01754486106573828
Peso: 2.4847665015936133
Altura: 2.199233004768637
Grupo Sanguíneo_A: 0.0
Grupo Sanguíneo_AB: 1.7412230178317802
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.8358701056169333
Fumante_Sim: 0.0
Nível de Atividade_Alto: 0.6961424797287861
Nível de Atividade_Baixo: 0.17642514217590116
Nível de Atividade_Moderado: 0.0


In [21]:
# Performance Regressão com LassoCV
performace_regressao(model_lasso_cv, X_test, y_test)

8.872879747160336

### Treinar Modelo de Regressão com Ridge (L2)

In [28]:
# Treinar o Modelo de regressão linear múltipla com Lasso
# Quanto maior o alpha, maior a penalização e mais coeficientes tendem a ser reduzios a zero
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [29]:
# Importância das Features - Lasso
importancia_features(model_ridge)

Importância das Features
Idade: 0.014856795381590114
Peso: 2.4863676267885966
Altura: 2.2052086366942296
Grupo Sanguíneo_A: 0.6068720485512523
Grupo Sanguíneo_AB: 1.2429953371312654
Grupo Sanguíneo_B: 0.8279752477160831
Grupo Sanguíneo_O: 0.1918519591557123
Fumante_Não: 1.0743689137577013
Fumante_Sim: 1.0743689137461385
Nível de Atividade_Alto: 1.4197372174686418
Nível de Atividade_Baixo: 0.42678204861783975
Nível de Atividade_Moderado: 0.992955168899216


In [ ]:
# Performance Regressão com Lasso
performace_regressao(model_ridge, X_test, y_test)

9.017609728696327

In [31]:
# Treinar com RidgeCV
model_ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1], cv=5)
model_ridge_cv.fit(X, y)

,alphas,"[0.1, 0.5, ...]"
,fit_intercept,True
,scoring,None
,cv,5
,gcv_mode,None
,store_cv_results,False
,alpha_per_target,False


In [ ]:
# Importância das Features - RidgeCV
importancia_features(model_ridge_cv)

Importância das Features
Idade: 0.016814725900195324
Peso: 2.4741983109179415
Altura: 2.190504813780751
Grupo Sanguíneo_A: 0.5897962408355004
Grupo Sanguíneo_AB: 1.6514464168680085
Grupo Sanguíneo_B: 0.4633210743381151
Grupo Sanguíneo_O: 0.5983291016990676
Fumante_Não: 1.1564340048585906
Fumante_Sim: 1.1564340048583872
Nível de Atividade_Alto: 0.8729024851208996
Nível de Atividade_Baixo: 0.646615518997282
Nível de Atividade_Moderado: 0.22628696612208432


In [33]:
# Performance Regressão com RidgeCV
performace_regressao(model_ridge_cv, X_test, y_test)

8.84949826711917